In [126]:
import pandas as pd, numpy as np

In [127]:
df = pd.read_parquet("data/uber_tripdata_2023-08.parquet")

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824209 entries, 0 to 2824208
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [129]:
# convert data type to datetime type
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824209 entries, 0 to 2824208
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [130]:
# clean data and extract values in the timestamp to create separate dimensions
datetime_df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)
datetime_df['pick_hour'] = datetime_df['tpep_pickup_datetime'].dt.hour
datetime_df['pick_day'] = datetime_df['tpep_pickup_datetime'].dt.day
datetime_df['pick_month'] = datetime_df['tpep_pickup_datetime'].dt.month
datetime_df['pick_year'] = datetime_df['tpep_pickup_datetime'].dt.year
datetime_df['pick_day'] = datetime_df['tpep_pickup_datetime'].dt.day
datetime_df['pick_weekday'] = datetime_df['tpep_pickup_datetime'].dt.weekday

datetime_df['drop_hour'] = datetime_df['tpep_dropoff_datetime'].dt.hour
datetime_df['drop_day'] = datetime_df['tpep_dropoff_datetime'].dt.day
datetime_df['drop_month'] = datetime_df['tpep_dropoff_datetime'].dt.month
datetime_df['drop_year'] = datetime_df['tpep_dropoff_datetime'].dt.year
datetime_df['drop_day'] = datetime_df['tpep_dropoff_datetime'].dt.day
datetime_df['drop_weekday'] = datetime_df['tpep_dropoff_datetime'].dt.weekday


In [131]:
# assign a unique id for table joins
datetime_df['datetime_id'] = datetime_df.index

In [132]:
# rearrange columns in dataframe
datetime_df = datetime_df[['datetime_id', 
               'tpep_pickup_datetime', 'pick_hour', 'pick_weekday', 'pick_year', 'pick_day', 'pick_month', 
               'tpep_dropoff_datetime', 'drop_hour', 'drop_weekday', 'drop_year', 'drop_day', 'drop_month']]
datetime_df

,datetime_id,tpep_pickup_datetime,pick_hour,pick_weekday,pick_year,pick_day,pick_month,tpep_dropoff_datetime,drop_hour,drop_weekday,drop_year,drop_day,drop_month
0,0,2023-08-01 00:26:44,0,1,2023,1,8,2023-08-01 00:45:25,0,1,2023,1,8
1,1,2023-08-01 00:55:42,0,1,2023,1,8,2023-08-01 01:00:53,1,1,2023,1,8
2,2,2023-08-01 00:32:04,0,1,2023,1,8,2023-08-01 01:09:03,1,1,2023,1,8
3,3,2023-08-01 00:13:37,0,1,2023,1,8,2023-08-01 00:41:15,0,1,2023,1,8
4,4,2023-08-01 00:24:46,0,1,2023,1,8,2023-08-01 00:41:31,0,1,2023,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792185,2792185,2023-08-31 23:40:14,23,3,2023,31,8,2023-08-31 23:56:46,23,3,2023,31,8
2792186,2792186,2023-08-31 23:15:42,23,3,2023,31,8,2023-08-31 23:36:57,23,3,2023,31,8
2792187,2792187,2023-08-31 23:28:52,23,3,2023,31,8,2023-08-31 23:52:00,23,3,2023,31,8
2792188,2792188,2023-08-31 23:59:58,23,3,2023,31,8,2023-09-01 00:13:01,0,4,2023,1,9


In [133]:
# add id to passenger_count and trip_distance tables and rearrange and update data type
passenger_count_df = df[['passenger_count']].drop_duplicates().reset_index(drop=True)
passenger_count_df['passenger_count_id'] = passenger_count_df.index
passenger_count_df = passenger_count_df[['passenger_count_id', 'passenger_count']]
passenger_count_df = passenger_count_df.dropna()
passenger_count_df['passenger_count'] = passenger_count_df['passenger_count'].astype(int)


trip_distance_df = df[['trip_distance']].drop_duplicates().reset_index(drop=True)
trip_distance_df['trip_distance_id'] = trip_distance_df.index
trip_distance_df = trip_distance_df[['trip_distance_id', 'trip_distance']]

In [134]:
passenger_count_df.head()

,passenger_count_id,passenger_count
0,0,1
1,1,2
2,2,5
3,3,3
4,4,4


In [135]:
trip_distance_df.head()

,trip_distance_id,trip_distance
0,0,4.30
1,1,0.00
2,2,20.61
3,3,13.37
4,4,3.20


In [136]:
# assign rate code name to table, clean NaN rows, update data type
rate_code_name = {1: 'Standard Rate', 2: 'JFK', 3: 'Newark', 4: 'Nassau or Westchester', 5: 'Negotiated fare', 6: 'Group ride'}

rate_code_df = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
rate_code_df['rate_code_id'] = rate_code_df.index
rate_code_df['rate_code_name'] = rate_code_df['RatecodeID'].map(rate_code_name)
rate_code_df = rate_code_df[['rate_code_id', 'RatecodeID', 'rate_code_name']]
rate_code_df = rate_code_df.dropna()
rate_code_df['RatecodeID'] = rate_code_df['RatecodeID'].astype(int)

rate_code_df.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard Rate
1,1,2,JFK
2,2,5,Negotiated fare
3,3,4,Nassau or Westchester
5,5,3,Newark


In [140]:
# assign payment code name to table, clean NaN rows, update data type
payment_code_name = {1: 'Credit Card', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Unknown', 6: 'Voided trip'}

payment_type_df = df[['payment_type']].drop_duplicates().reset_index(drop=True)
payment_type_df['payment_type_id'] = payment_type_df.index
payment_type_df['payment_type_name'] = payment_type_df['payment_type'].map(payment_code_name)
payment_type_df = payment_type_df[['payment_type_id', 'payment_type', 'payment_type_name']]
payment_type_df = payment_type_df.dropna()

payment_type_df.head()

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit Card
1,1,3,No charge
2,2,2,Cash
3,3,4,Dispute


In [143]:
fact_table = df.merge(passenger_count_df, on='passenger_count') \
            .merge(trip_distance_df, on='trip_distance') \
            .merge(rate_code_df, on='RatecodeID') \
            .merge(payment_type_df, on='payment_type') \
            .merge(datetime_df, on=['tpep_pickup_datetime', 'tpep_dropoff_datetime']) \
            [[
                'VendorID', 'datetime_id', 'passenger_count_id', 'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
                'PULocationID', 'DOLocationID', 'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                'improvement_surcharge', 'total_amount'
            ]]
            
fact_table

,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,PULocationID,DOLocationID,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,0,0,0,0,N,263,90,0,21.9,3.50,0.5,5.35,0.00,1.0,32.25
1,2,884,0,0,0,N,132,219,0,20.5,1.00,0.5,4.95,0.00,1.0,29.70
2,1,2713,0,0,0,N,48,226,0,20.5,3.50,0.5,5.10,0.00,1.0,30.60
3,1,4990,0,0,0,N,231,162,0,26.1,2.50,0.5,6.00,0.00,1.0,36.10
4,1,7537,0,0,0,N,143,13,0,19.8,2.50,0.5,3.00,0.00,1.0,26.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2716130,2,1657183,0,2675,3,N,132,265,1,124.1,0.00,0.5,0.00,8.60,1.0,135.95
2716131,2,2364056,0,4141,3,N,132,265,1,-158.4,-1.00,0.0,0.00,-19.69,-1.0,-181.84
2716132,2,2364056,0,4141,3,N,132,265,1,158.4,1.00,0.0,0.00,19.69,1.0,181.84
2716133,1,2672466,0,4711,3,N,132,265,1,532.2,4.25,0.5,0.00,0.00,1.0,537.95
